In [1]:
import sympy as sp

sp.init_printing()

In [11]:
# Utility functions

def make_symbol(name, upper=None, lower=None, expr=None, nonnegative=True, **kwargs):
    if upper:
        name += "^{%s}" % upper
    if lower:
        name += "_{%s}" % lower
    sym = sp.Symbol(name, nonnegative=nonnegative, **kwargs)
    
    if expr:
        return sp.Eq(sym, expr, evaluate=True)
    else:
        return sym
    

def make_concentration(name, valence=0, label=None, **kwargs):
    if valence > 0:
        sign = "+"
    elif valence < 0:
        sign = "-"
    else:
        sign = ""
        
    name = "[%s^{%d%s}]" % (name, valence, sign) 
    return make_symbol(name, upper=None, lower=label, expr=None, nonnegative=True, **kwargs)

In [12]:
make_concentration('Ca', +2, 'mito')